## 用朴素贝叶斯完成语种检测
by 寒小阳(hanxiaoyang.ml@gmail.com)

#### 文本分类
* 预处理(清洗、分词...)
* 对文本做表示(向量化的表示) 【TF-IDF，词袋，word2vec，word embedding...】
* 建模(机器学习、深度学习) 【朴素贝叶斯、SVM、TextCNN、TextRNN...】

朴素贝叶斯 = 贝叶斯公式 + 条件独立假设

+ 平滑(拉普拉斯平滑)

我们试试用朴素贝叶斯完成一个语种检测的分类器，说起来，用朴素贝叶斯完成这个任务，其实准确度还不错。

![](./detector.png)
![](./detector2.png)

机器学习的算法要取得好效果，离不开数据，咱们先拉点数据（twitter数据，包含English, French, German, Spanish, Italian 和 Dutch 6种语言）瞅瞅。

In [1]:
!head -5 data.csv

1 december wereld aids dag voorlichting in zuidafrika over bieten taboes en optimisme,nl
1 millón de afectados ante las inundaciones en sri lanka unicef está distribuyendo ayuda de emergencia srilanka,es
1 millón de fans en facebook antes del 14 de febrero y paty miki dani y berta se tiran en paracaídas qué harías tú porunmillondefans,es
1 satellite galileo sottoposto ai test presso lesaestec nl galileo navigation space in inglese,it
10 der welt sind bei,de


In [2]:
in_f = open('data.csv')
lines = in_f.readlines()
in_f.close()
dataset = [(line.strip()[:-3], line.strip()[-2:]) for line in lines]

了解一下你的数据

In [3]:
dataset[:5]

[('1 december wereld aids dag voorlichting in zuidafrika over bieten taboes en optimisme',
  'nl'),
 ('1 mill\xc3\xb3n de afectados ante las inundaciones en sri lanka unicef est\xc3\xa1 distribuyendo ayuda de emergencia srilanka',
  'es'),
 ('1 mill\xc3\xb3n de fans en facebook antes del 14 de febrero y paty miki dani y berta se tiran en paraca\xc3\xaddas qu\xc3\xa9 har\xc3\xadas t\xc3\xba porunmillondefans',
  'es'),
 ('1 satellite galileo sottoposto ai test presso lesaestec nl galileo navigation space in inglese',
  'it'),
 ('10 der welt sind bei', 'de')]

为了一会儿检测一下咱们的分类器效果怎么样，我们需要一份测试集。

所以把原数据集分成训练集的测试集，咱们用sklearn自带的分割函数。

In [4]:
from sklearn.model_selection import train_test_split
x, y = zip(*dataset)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

In [7]:
len(x_train)

6799

模型要有好效果，数据质量要保证

我们用正则表达式，去掉噪声数据

In [10]:
import re

def remove_noise(document):
    noise_pattern = re.compile("|".join(["http\S+", "\@\w+", "\#\w+"]))
    clean_text = re.sub(noise_pattern, "", document)
    return clean_text.strip()

remove_noise("Trump images are now more popular than cat gifs. @trump #trends http://www.trumptrends.html")

'Trump images are now more popular than cat gifs.'

下一步要做的就是在降噪数据上抽取出来有用的特征啦，我们抽取1-gram和2-gram的统计特征

### 文本分类 = 文本表示+分类模型
#### 文本表示：BOW/N-gram/TF-IDF/word2vec/word embedding
#### 分类模型：NB/LR/SVM/CNN/RNN(LSTM)

语种判断：拉丁语系，都是字母构成，甚至大部分字母都一样，怎么判断？ => 字母的使用(顺序和频度)不一样

#### 文本表示
① 分词(NLPIR, jieba)<br>
第1句话：\[w1, w2, w3, w15, ...\]<br>
第2句话：\[w3, w17, w31, w45, ...\]<br>
第3句话：\[w13, w27, w131, w245, ...\]<br>

② 词频统计：<br>
w3 count3<br>
w7 count7<br>
wi count_i<br>
...

③ 构建词典：<br>
选出频次最高的N个词<br>
开1\*N的向量空间，每个位置是一个词<br>

④ 映射：把每句话共同构建的词典进行映射(基本版的词袋模型)<br>
第1句话：\[1,0,1,0,1,0,0...\]<br>
第2句话：\[0,0,1,1,0,0,1...\]<br>
第3句话：\[0,1,1,0,0,0,1...\]<br>
...

**词袋模型**：<br>
"李雷喜欢韩梅梅" => ("李雷","喜欢","韩梅梅")<br>
"韩梅梅喜欢李雷" => ("李雷","喜欢","韩梅梅")<br>


⑤-1 N-gram （语序）<br>
2-gram 二元语言模型<br>
"李雷喜欢韩梅梅" => ("李雷","喜欢","韩梅梅","李雷喜欢", "喜欢韩梅梅")<br>
"韩梅梅喜欢李雷" => ("李雷","喜欢","韩梅梅","韩梅梅喜欢","喜欢李雷")<br>

⑤-2 TF-IDF（权重）

⑥ 离散表示形式 => 连续表示形式<br>

宾馆 = 酒店 = 旅店 = 旅馆 （相近）<br>

WordNet：（上位词、下位词、近义词...）<br>
喜欢 = 稀罕 = 中意<br>
666<br>

word2vec 把每个词映射为稠密词向量，关联语义的词它们的词向量比较接近。<br>
1\*300词向量

Trump images are now more popular than cat gifs

计算机读不懂文本表述，怎么办？<br>
回想一下，我们是怎么阅读的？

语种判断：拉丁语系，字母组成的，甚至字母也一样 => 字母的使用(次序、频次)不一样

**词袋模型**："李雷喜欢韩梅梅" => ("李雷","喜欢","韩梅梅")<br>
"韩梅梅喜欢李雷" => ("李雷","喜欢","韩梅梅")<br>
"李雷喜欢韩梅梅" => ("李雷","喜欢","韩梅梅","李雷喜欢", "喜欢韩梅梅")<br>
"韩梅梅喜欢李雷" => ("李雷","喜欢","韩梅梅","韩梅梅喜欢","喜欢李雷")<br>
trump images are now more popular than cat gifs => t(5) r(4) u(3) m(6) p(3)

给每个出现的最小粒度的元素，都给了一个编码<br>
(123 24 56 1024 1567)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfVectorizer

vec = CountVectorizer(
    lowercase=True,     # 英文文本全小写
    analyzer='char_wb', # 逐个字母解析
    ngram_range=(1,2),  # 1=出现的字母以及每个字母出现的次数，2=出现的连续2个字母，和连续2个字母出现的频次
    # trump images are now... => 1gram = t,r,u,m,p... 2gram = tr,ru,um,mp...
    max_features=1000,  # keep the most common 1000 ngrams
    preprocessor=remove_noise
)
vec.fit(x_train)

def get_features(x):
    vec.transform(x)

In [12]:
result = vec.transform(["Trump images are now more popular than cat gifs"])

In [13]:
result

<1x1000 sparse matrix of type '<type 'numpy.int64'>'
	with 60 stored elements in Compressed Sparse Row format>

把分类器import进来并且训练

In [13]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

看看我们的准确率如何

In [14]:
classifier.score(vec.transform(XTest), yTest)

0.9770621967357741

能在1500句话上，训练得到准确率97.7%的分类器，效果还是不错的。

如果大家加大语料，准确率会非常高。

### 规范化，写成一个class

In [16]:
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


class LanguageDetector():

    def __init__(self, classifier=MultinomialNB()):
        self.classifier = classifier
        self.vectorizer = CountVectorizer(ngram_range=(1,2), max_features=1000, preprocessor=self._remove_noise)

    def _remove_noise(self, document):
        noise_pattern = re.compile("|".join(["http\S+", "\@\w+", "\#\w+"]))
        clean_text = re.sub(noise_pattern, "", document)
        return clean_text

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

In [20]:
in_f = open('data.csv')
lines = in_f.readlines()
in_f.close()
dataset = [(line.strip()[:-3], line.strip()[-2:]) for line in lines]
x, y = zip(*dataset)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

language_detector = LanguageDetector()
language_detector.fit(x_train, y_train)
print(language_detector.predict('This is an English sentence'))
print(language_detector.score(x_test, y_test))

['en']
0.977062196736
